# 🏋️ PyTorch Compilation Exercises

The following exercises are designed to help you identify and fix common performance pitfalls when using `torch.compile`. `torch.compile` is the recommended way to speed up PyTorch code by compiling the code into optimised kernels.

For each exercise the aim is:
* Run the code
* Use the logs to debug why the code might be sub-optimal [useful docs](https://docs.pytorch.org/tutorials/intermediate/torch_compile_tutorial.html)
* Look into fixing and optimising the code
* Re-run the code and validate fixes

Along the way both `%timeit` and the [Pytorch profiler](https://docs.pytorch.org/tutorials/recipes/recipes/profiler_recipe.html#using-tracing-functionality) might be useful or interesting to see the impact and whats happening.

In [1]:
import torch

This logging is what we are going to use to find some of our issues and maybe even provide some hints on how to fix them.

In [2]:
torch._logging.set_logs(graph_code=True, recompiles=True, graph_breaks=True)

## Exercise 1

In [ ]:
@torch.compile
def add_val(x, val):
    return x + val

for i in range(5):
    x = torch.randn(10)
    print(add_val(x, i))

In [ ]:
#@title Exercise 1 Solution
"""
Issue - excessive recompilation

When you pass a Python scalar (like an `int` or `float`) to a compiled function,
PyTorch often treats it as a constant to optimize the graph. If that value changes
frequently, PyTorch must recompile the function every time, which is very slow.

Note - here PyTorch has been clever and is only recompiling once but in more
complex cases it may not be able to do this and also that for larger graphs compile
times can be significant so even compiling twice can have a negative impact.

Solution - pass in the value as a Tensor so it gets converted to a Tracer on
the first iteration.
"""
@torch.compile
def add_val_solution(x, val):
    return x + val

for i in range(5):
    x = torch.randn(10)
    print(add_val_solution(x, torch.Tensor([i])))

## Exercise 2

In [ ]:
import scipy

@torch.compile
def scipy_interaction(x):
    x_np = x.detach().numpy()
    y_np = scipy.special.expit(x_np)
    return torch.from_numpy(y_np)

x = torch.randn(100)
res = scipy_interaction(x)

In [ ]:
#@title Exercise 2 Solution
"""
Removing Graph Breaks

Interacting with non-PyTorch libraries (like SciPy) inside a compiled function
forces PyTorch to introduce a `graph break`. The downsides of breaking the graph
is that this code can no longer be optimised and runs in the Python, in particular
if running on accelerators the impact of this can be greater than you will see here.

Solution - Modify `scipy_interaction` to use pure PyTorch operations
(e.g., `torch.special.expit`) to avoid the graph break.

NOTE: This doesn't happen with most numpy ops as pytorch is able to automatically
compile numpy code into the pytorch graph - the only difference can come in the
way scalars and types are treated. See
https://pytorch.org/blog/compiling-numpy-code/ for more details
"""
@torch.compile
def scipy_interaction_solution(x):
    return torch.special.expit(x)

x = torch.randn(100)
res = scipy_interaction_solution(x)

## Exercise 3

In [ ]:
@torch.compile
def debugging_code(x, y):
  x+= 2.
  print(x.shape)
  return x @ y  # equivalent of torch.matmul

debugging_code(torch.randn(6, 10), torch.randn(10, 8))

In [ ]:
#@title Execise 3 Solution

"""Here the fix is simple to remove any debugging code that might be left.

And a key thing is to keep an eye on this when writing code in future - although
things like this and putting in graph breaks are useful for debugging when
the code isn't working as expected.
"""

## Exercise 4

In [ ]:
@torch.compile
def greater_than(x):
  if x > 5.:
    return x + 2.
  else:
    return torch.ones_like(x)

print(greater_than(torch.tensor(3.)))

In [ ]:
print(greater_than(torch.tensor(6.)))

In [ ]:
#@title Exercise 4 solution
"""
Here we get a graph break because of control flow and actually we can use the hint
that is given which tells us that we can avoid it by using pytorch's own control
flow which has the benefit that it can be traced through.

From the traced graph we can see that we actually get three graphs, two the first
time we run the code and then a third when we take the other branch of the if
statement. These are then orchestrated in Python.

Soultion - switch the code to use `torch.cond`

NOTE - think about using `timeit` example in the cell below - to look at how the
performance changes
"""
@torch.compile
def greater_than_solution(x):
  # Note: Use .clone() because torch.cond prohibits aliasing input to output
  output = torch.cond(torch.all(x > 5.), lambda x: x + 2., lambda x: torch.ones_like(x), (x,))
  return output

print(greater_than_solution(torch.tensor(3.)))
print(greater_than_solution(torch.tensor(6.)))

In [ ]:
#@title Investigating exercise 4

"""
One thing that can be useful and interesting when looking at code is basic timing
execises and in colab the magic timeit can be very helpful to do this. In particular
one thing this does is to run your code.
"""
%timeit greater_than(torch.tensor(3.))
%timeit greater_than_solution(torch.tensor(3.))

## Exercise 5

In [ ]:
@torch.compile
def custom_relu(x):
    out = torch.empty_like(x)
    for i in range(len(x)):
        if x[i] > 0:
            out[i] = x[i]
        else:
            out[i] = 0.0
    return out

print("\nExercise 3: optimizing control flow...")
x = torch.randn(100)
res = custom_relu(x)

In [ ]:
#@title Exercise 5 Solution
"""
Vectorizing Control Flow

Python loops and `if` statements over tensor data are extremely slow and cause
graph breaks (or full graph capture failure) because the compiler can't easily
predict the path.

Soultion - change the code to use `torch.where` and element-wise operations to
perform the same operation without needing a loop.
"""
@torch.compile
def custom_relu_solution(x):
  mask = torch.gt(x, 0.)
  return torch.where(mask, x, torch.zeros_like(x))

@torch.compile
def custom_relu_oneliner(x):
  return torch.clamp(x, min=0.)

x = torch.randn(100)
res = custom_relu_solution(x)

## Exercise 6

In [ ]:
@torch.compile
def batch_multiply(xs, y):
  return [torch.matmul(x, y) for x in xs]

xs = torch.rand([128, 64, 32])
y = torch.rand([32, 128])

result = batch_multiply(xs, y)

In [ ]:
#@title Exercise 6 solution

"""
Many ops treat leading dimensions as batch dimensions and so therefore a simple
matmul can be used with the dimension looped across treated in the same way.
Look at the size of the graph generated and also the difference in time taken
to perform the computation.

Solution - instead of using the loop just allow the computation to happen within
a single operation.

NOTE - the solution here is simple but it could be more complex depending on the
dimensions of the tensors involved, in these cases making sure that the right
operations are happening can be harder but is important.
"""

@torch.compile
def batch_multiply_solution(xs, y):
  return torch.matmul(xs, y)

res = batch_multiply_solution(xs, y)
%timeit batch_multiply_solution(xs, y)

## Exercise 7

In this exercise using the profiler tool to generate profiles which can then be loaded into Chrome trace viewer (`chrome://tracing`) will be very useful in finding the problems and the solution.

In [10]:
#@title Dataset loading code

import asyncio

from torch.utils.data import Dataset, DataLoader

class DummyDataset(Dataset):
    def __init__(self, size=1000):
        self.size = size
        self.data = torch.randn(size, 128)
        self.labels = torch.randn(size, 1)

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        # Simulate slow data loading (e.g., reading from file system/network)
        time.sleep(0.001)
        return self.data[idx], self.labels[idx]

In [11]:
import time
from torch.profiler import profile, record_function, ProfilerActivity

model = torch.nn.Sequential(
    torch.nn.Linear(128, 1024),
    torch.nn.ReLU(),
    torch.nn.Linear(1024, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 1)
)
if torch.cuda.is_available():
    model = model.cuda()

dataset = DummyDataset()
dataloader = DataLoader(dataset, batch_size=32, num_workers=0)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

start_time = time.monotonic()
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
  for epoch in range(2):
      for batch_idx, (data, target) in enumerate(dataloader):
          if torch.cuda.is_available():
              data, target = data.cuda(), target.cuda()
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()
print(f"Training finished in {time.monotonic() - start_time:.2f} seconds.")

prof.export_chrome_trace("trace.json")

/usr/local/lib/python3.12/dist-packages/torch/autograd/profiler.py:271: UserWarning: CUDA is not available, disabling CUDA profiling
  warn("CUDA is not available, disabling CUDA profiling")


Training finished in 2.68 seconds.


In [ ]:
#@title Solution Exercise 7

"""
In this case we are input bound, we can see from the trace that most of the
time is spent in the dataset loading instead of the computation.

If you look into the dataset we have put in hangs to simulate what happens when
reading from disk etc. In the `DataLoader` used we were using no workers which
means that all the data loading was happening in the main thread. Instead you
can use workers which means that this happens in the background. Try playing
around with the num_workers and see what the impact is.

See https://docs.pytorch.org/tutorials/beginner/basics/data_tutorial.html for
more details on working with data.
"""
dataloader_fast = DataLoader(dataset, batch_size=32, num_workers=8)

start_time = time.time()
with profile(activities=[ProfilerActivity.CPU]) as prof:
  for epoch in range(2):
      for batch_idx, (data, target) in enumerate(dataloader_fast):
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          loss.backward()
          optimizer.step()

print(f"Training finished in {time.time() - start_time:.2f} seconds.")

prof.export_chrome_trace("trace.json")

### Exercise 7 extension

Currently none of this code is compiled

Where should you add a `torch.compile`? What is the impact of this?

In [ ]:
#@title Exercise 7 extension solution

"""One thing to note here is that the time taken is initially much longer.

What do you notice when looking at the trace?
This is the compile time - when running it the second time you see much more
the speed-ups expected.
"""

dataloader_fast = DataLoader(dataset, batch_size=32, num_workers=8)

@torch.compile
def forward(data, target):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

start_time = time.time()
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
  for epoch in range(2):
      for batch_idx, (data, target) in enumerate(dataloader_fast):
          forward(data, target)

print(f"Fast training finished in {time.time() - start_time:.2f} seconds.")

prof.export_chrome_trace("trace.json")